In [7]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))



df = pd.read_csv(os.path.join(os.getcwd(), "../../data/total_4am_with_hashrate.csv"), index_col=0)


In [8]:
from performance import get_performance


df["HR_MA_30"] = df["Hashrate (TH/s)"].rolling(window=30).mean()
df["HR_MA_60"] = df["Hashrate (TH/s)"].rolling(window=60).mean()

# Implement RSI strategy for long positions only
df['signal'] = 0  # Default to no position
for i in range(26, len(df)):
    # Buy condition
    if df.loc[i, "HR_MA_30"] >= df.loc[i, "HR_MA_60"] and df.loc[i-1, "HR_MA_30"] < df.loc[i-1, "HR_MA_60"]:
        df.loc[i, 'signal'] = 1
    # Sell condition
    elif df.loc[i, "HR_MA_30"] < df.loc[i, "HR_MA_60"] and df.loc[i-1, "HR_MA_30"] >= df.loc[i-1, "HR_MA_60"]:
        df.loc[i, 'signal'] = -1

# Manage positions
df['position'] = 0
holding_position = False

for i in range(1, len(df)):
    if df['signal'].iloc[i] == 1 and not holding_position:
        df.loc[i, 'position'] = 1
        holding_position = True
    elif df['signal'].iloc[i] == -1 and holding_position:
        df.loc[i, 'position'] = 0
        holding_position = False
    else:
        df.loc[i, 'position'] = df['position'].iloc[i-1]

# Calculate the strategy returns (only when in a long position)
df['strategy_returns'] = df['position'].shift(1) * df['close'].pct_change()

df['strategy_returns2'] = df['strategy_returns']

for i in range(1, len(df)):
    buy_price = df.loc[i-1, "close"]
    buy_price_copy = buy_price
    sell_price = df.loc[i, "close"]
    sell_price_copy = sell_price
    if df.loc[i-1, 'position'] == 1 and df.loc[i-1, 'signal'] == 1:
        # df.loc[i, 'strategy_returns2'] = (df.loc[i,'position'])/(df.loc[i-1, 'position'] * 1.002) -1
        buy_price = df.loc[i-1, "close"] * 1.002
    if df.loc[i, 'position'] == 0 and df.loc[i-1, 'position'] != 0:
        # df.loc[i, 'strategy_returns2'] = (df.loc[i,'position'] * 0.998)/(df.loc[i-1, 'position']) -1
        sell_price = df.loc[i, "close"] * 0.998
    
    if buy_price == buy_price_copy and sell_price == sell_price_copy:
        continue


    df.loc[i, "strategy_returns2"] = sell_price/buy_price - 1

df['cumulative_returns'] = (1 + df['strategy_returns']).cumprod()

df["cumulative_returns2"] = (1 + df['strategy_returns2']).cumprod()


# Calculate the benchmark cumulative returns (buy and hold strategy)
df['benchmark_returns'] = (1 + df['close'].pct_change()).cumprod()

# Save the results to a CSV file
df.to_csv(f"../../results/hashrate/3060_basic.csv")

# Get performance metrics and plot the results
get_performance(df=df, title=f"3060_basic")


======Investment Summary======
Strategy        : 3060_basic
total_return    : 70.14
cagr            : 19.17
mdd             : 64.19
total_return_w_fee : 64.45
cagr_w_fee      : 17.84
mdd_w_fee       : 64.62
investing_days  : 1106
